# EDA

In [19]:
import pandas as pd
import numpy as np
from etl import safe_literal_eval, dtype_checker, movies_df

In [4]:
movies = pd.read_parquet("parquet_data/movies_parquet.parquet")
credits = pd.read_parquet("parquet_data/credits_parquet.parquet")

### Let's recover the fields vote_count and voten average from the CSV movie file.

In [21]:
movies_csv = pd.read_csv("data/movies_dataset.csv")

C:\Users\jdieg\AppData\Local\Temp\ipykernel_13732\3445815435.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_csv = pd.read_csv("data/movies_dataset.csv")


In [53]:
movies_df = movies_df.merge(movies_csv[["id","vote_count", "vote_average"]], on="id", how="left")

### Missing Data

In [64]:
movies_df.isna().sum()

collection_id           40944
budget                      0
id                          0
original_language          11
production_countries        0
release_date                0
revenue                     0
runtime                   246
spoken_languages            0
title                       0
release_year                0
return_on_investment        0
vote_count                  0
vote_average                0
dtype: int64

Having a look at original language field

In [67]:
movies_df[movies_df["original_language"].isna()].head()

,collection_id,budget,id,original_language,production_countries,release_date,revenue,runtime,spoken_languages,title,release_year,return_on_investment,vote_count,vote_average
19595,NaN,0.0,283101,None,"[{'iso_3166_1': 'AT', 'name': 'Austria'}, {'is...",2004-10-11,0.0,95.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",Shadowing the Third Man,2004,0.0,0.0,0.0
21621,NaN,0.0,103902,None,"[{'iso_3166_1': 'AU', 'name': 'Australia'}]",2007-08-04,0.0,94.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Unfinished Sky,2007,0.0,8.0,6.4
22849,NaN,0.0,359195,None,"[{'iso_3166_1': 'US', 'name': 'United States o...",1960-03-31,0.0,69.0,"[{'iso_639_1': 'en', 'name': 'English'}]",13 Fighting Men,1960,0.0,0.0,0.0
32155,NaN,0.0,147050,None,"[{'iso_3166_1': 'US', 'name': 'United States o...",1929-10-31,0.0,8.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Lambchops,1929,0.0,2.0,6.5
37410,NaN,0.0,257095,None,"[{'iso_3166_1': 'CZ', 'name': 'Czech Republic'}]",1950-01-26,0.0,87.0,"[{'iso_639_1': 'cs', 'name': 'Český'}]",Prince Bayaya,1950,0.0,1.0,5.0


And those rows with null values in field original_language also has missing values on other field sudh as vote_count, vote_average, etc.. having 3 features missing is a significant for our EDa.. so that the more suitable approach is drop them

In [70]:
movies_df.dropna(subset=["original_language"], inplace=True)

### runtime feature missing values

In [79]:
movies_df[movies_df["runtime"].isna()].head()

,collection_id,budget,id,original_language,production_countries,release_date,revenue,runtime,spoken_languages,title,release_year,return_on_investment,vote_count,vote_average
634,NaN,0.0,287305,de,"[{'iso_3166_1': 'DE', 'name': 'Germany'}]",1996-03-21,0.0,NaN,"[{'iso_639_1': 'Unknown', 'name': 'Unknown'}]",Peanuts – Die Bank zahlt alles,1996,0.0,1.0,4.0
635,NaN,0.0,339428,de,"[{'iso_3166_1': 'DE', 'name': 'Germany'}]",1996-03-14,65335.0,NaN,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Happy Weekend,1996,0.0,0.0,0.0
644,NaN,0.0,278978,de,"[{'iso_3166_1': 'DE', 'name': 'Germany'}]",1996-02-29,0.0,NaN,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Und keiner weint mir nach,1996,0.0,0.0,0.0
801,NaN,0.0,282919,de,"[{'iso_3166_1': 'DE', 'name': 'Germany'}]",1996-06-20,0.0,NaN,"[{'iso_639_1': 'Unknown', 'name': 'Unknown'}]",Diebinnen,1996,0.0,1.0,4.0
863,NaN,0.0,253632,fr,"[{'iso_3166_1': 'Unknown', 'name': 'Unknown'}]",1988-10-08,0.0,NaN,"[{'iso_639_1': 'Unknown', 'name': 'Unknown'}]",Baton Rouge,1988,0.0,0.0,0.0


Some features like revenue, budget, in consecuents roi (shor for return_on_investment), and some vote_count values in zero. Since this is few missing data, the more suitable approach is dropping those rows.

In [81]:
movies_df.dropna(subset=["runtime"], inplace=True)

In [83]:
movies_df.isna().sum()

collection_id           40694
budget                      0
id                          0
original_language           0
production_countries        0
release_date                0
revenue                     0
runtime                     0
spoken_languages            0
title                       0
release_year                0
return_on_investment        0
vote_count                  0
vote_average                0
dtype: int64

### collection_id feature
collection_id features is an exception here since this refers to more information about certain movies that are related to other ones (there's another dataset from etl that can be imported to merge on this id). A more appropriate feature would be adding a new feture that only indicates if it bellongs to a collection.

In [84]:
movies_df

,collection_id,budget,id,original_language,production_countries,release_date,revenue,runtime,spoken_languages,title,release_year,return_on_investment,vote_count,vote_average
0,10194,30000000.0,862,en,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Toy Story,1995,12.451801,5415.0,7.7
1,NaN,65000000.0,8844,en,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Jumanji,1995,4.043035,2413.0,6.9
2,119050,0.0,15602,en,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Grumpier Old Men,1995,0.000000,92.0,6.5
3,NaN,16000000.0,31357,en,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Waiting to Exhale,1995,5.090760,34.0,6.1
4,96871,0.0,11862,en,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Father of the Bride Part II,1995,0.000000,173.0,5.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45433,NaN,0.0,30840,en,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1991-05-13,0.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Robin Hood,1991,0.000000,26.0,5.7
45434,NaN,0.0,111109,tl,"[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Century of Birthing,2011,0.000000,3.0,9.0
45435,NaN,0.0,67758,en,"[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Betrayal,2003,0.000000,6.0,3.8
45436,NaN,0.0,227506,en,"[{'iso_3166_1': 'RU', 'name': 'Russia'}]",1917-10-21,0.0,87.0,"[{'iso_639_1': 'Unknown', 'name': 'Unknown'}]",Satan Triumphant,1917,0.000000,0.0,0.0


# Hipothesys

I would like to know how predominant the language english is in the movies. Parhaps thet would be an outlier

In [57]:
movies_df["original_language"]

array(['en', 'fr', 'zh', 'it', 'fa', 'nl', 'de', 'cn', 'ar', 'es', 'ru',
       'sv', 'ja', 'ko', 'sr', 'bn', 'he', 'pt', 'wo', 'ro', 'hu', 'cy',
       'vi', 'cs', 'da', 'no', 'nb', 'pl', 'el', 'sh', 'xx', 'mk', 'bo',
       'ca', 'fi', 'th', 'sk', 'bs', 'hi', 'tr', 'is', 'ps', 'ab', 'eo',
       'ka', 'mn', 'bm', 'zu', 'uk', 'af', 'la', 'et', 'ku', 'fy', 'lv',
       'ta', 'sl', 'tl', 'ur', 'rw', 'id', 'bg', 'mr', 'lt', 'kk', 'ms',
       'sq', None, 'qu', 'te', 'am', 'jv', 'tg', 'ml', 'hr', 'lo', 'ay',
       'kn', 'eu', 'ne', 'pa', 'ky', 'gl', 'uz', 'sm', 'mt', 'hy', 'iu',
       'lb', 'si'], dtype=object)

In [62]:
movies_df["spoken_languages"].apply(lambda languages: "en" in (language["iso_639_1"] for language in languages)).sum()

28765